In [1]:
# bring in the dependencies
from itertools import product, combinations
from dataclasses import dataclass
from typing import Optional, List, Dict, Iterable
from pprint import pprint
import os, sys

# project dependencies
proj_path = 'd:/code/Projects/property_mapping_sql_generator/'
sys.path.insert(0, proj_path)
from dsl.engine import *

In [2]:
# create the logic branch type i.e. a branch from a case statement
@dataclass(frozen=True)
class LogicBranch():
    result: str
    search_expr: Optional[str]
    ref: Optional[str]

In [3]:
# spike for case statement to support return of logic branches
class AltCase(Case):
    
    def get_branches(self) -> List[LogicBranch]:
        for src, res in self.search_result_pairs:
            yield LogicBranch(res.val, src.val, self.ref_field)
        yield LogicBranch(self.default.val, None, self.ref_field)
        
    def __repr__(self):
        pairs_str = ", ".join([f"({s.val}, {r.val})" for s, r in self.search_result_pairs])
        return f"AltCase({self.ref_field}, [{pairs_str}], default={self.default.val})"
    
    def __str__(self):
        return self.__repr__()

In [4]:
# example alt cases
p_case = AltCase('ref', [('x', 'p1'), ('y', 'p2')], default='pN')
list(p_case.get_branches())

[LogicBranch(result='p1', search_expr='x', ref='a.ref'),
 LogicBranch(result='p2', search_expr='y', ref='a.ref'),
 LogicBranch(result='pN', search_expr=None, ref='a.ref')]

In [5]:
a_case = AltCase('ref', [('x', 'a1'), ('y', 'a2'), ('z', 'a3')], default='aN')
list(a_case.get_branches())

[LogicBranch(result='a1', search_expr='x', ref='a.ref'),
 LogicBranch(result='a2', search_expr='y', ref='a.ref'),
 LogicBranch(result='a3', search_expr='z', ref='a.ref'),
 LogicBranch(result='aN', search_expr=None, ref='a.ref')]

In [6]:
# assume inline mapping for others e.g.
p = Property(p_case)
a = Analysis(a_case)
i = NoIndicator()
u = NoUom()
r = NoRatio()

In [7]:
from collections import defaultdict

me_rc : Dict[str, List[Mapping]] = defaultdict(list)  # mapping_el_by_ref_col
for e in [p, a, i, u, r]:
    key = None if e.is_inline() else e.mapping_func.ref_field
    me_rc[key].append(e)
    
pprint(me_rc)

defaultdict(<class 'list'>,
            {None: [NoIndicator(mapping_func='na', has_value=False),
                    NoUom(mapping_func='n/a', has_value=False),
                    NoRatio(mapping_func='NoRatio', has_value=False)],
             'a.ref': [Property(mapping_func=AltCase(a.ref, [(x, p1), (y, p2)], default=pN), has_value=True),
                       Analysis(mapping_func=AltCase(a.ref, [(x, a1), (y, a2), (z, a3)], default=aN), has_value=True)]})


In [8]:
# in real version this should be exposed via the `Mapping` type
def get_branches(e: Mapping) -> List[LogicBranch]:
    if isinstance(e.mapping_func, str):
        yield LogicBranch(e.mapping_func, None, None)
    elif e.is_inline():
        yield LogicBranch(e.mapping_func.val, None, None)
    elif isinstance(e.mapping_func, AltCase):
        for b in e.mapping_func.get_branches():
            yield b
    else:
        raise NotImplementedError(f'Only str, inline or AltCase supported, got type: {type(e)}')

In [9]:
# get valid combs of logic branches
lb_rc: Dict[str, List[LogicBranch]] = defaultdict(list)  # logic_br_by_ref_col

for ref, elements in me_rc.items():
    
    print(f"\n{'='*80}")
    print(f"\nprocessing ref: {ref}")
    
    # get unique search terms
    lb_s: Dict[str, List[LogicBranch]] = defaultdict(list) # logic br by srch
        
    # get possible logic branch combinations) 
    for e in elements:
        print(f"\nel func: {e.mapping_func}:")
        for b in get_branches(e):
            pprint(b)
             #lb_s[b.search_expr].append(b)
    
    # get all unique search terms (for current ref field)
    search_terms = set([lb.search_expr for e in elements for lb in get_branches(e)])

    print(f"\nunique search terms: {search_terms}")




processing ref: a.ref

el func: AltCase(a.ref, [(x, p1), (y, p2)], default=pN):
LogicBranch(result='p1', search_expr='x', ref='a.ref')
LogicBranch(result='p2', search_expr='y', ref='a.ref')
LogicBranch(result='pN', search_expr=None, ref='a.ref')

el func: AltCase(a.ref, [(x, a1), (y, a2), (z, a3)], default=aN):
LogicBranch(result='a1', search_expr='x', ref='a.ref')
LogicBranch(result='a2', search_expr='y', ref='a.ref')
LogicBranch(result='a3', search_expr='z', ref='a.ref')
LogicBranch(result='aN', search_expr=None, ref='a.ref')

unique search terms: {'y', 'z', 'x', None}


processing ref: None

el func: na:
LogicBranch(result='na', search_expr=None, ref=None)

el func: n/a:
LogicBranch(result='n/a', search_expr=None, ref=None)

el func: NoRatio:
LogicBranch(result='NoRatio', search_expr=None, ref=None)

unique search terms: {None}
